In [6]:
# Challenge Sorbonne - DST
#
import numpy as np
import pandas as pd
import os
import os
#import rarfile
import networkx as nx
import io
import re
import random
import csv
from datetime import datetime
import tqdm as notebook_tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, Subset, Dataset, DataLoader
from torch.optim import AdamW
#!pip install torch_geometric
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.nn import GATConv, global_mean_pool
import networkx as nx

from collections import defaultdict, Counter

import matplotlib.pyplot as plt
import plotly.io as pio
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

from sklearn.preprocessing import LabelEncoder, RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import tqdm as notebook_tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device is", device)

current_directory = os.getcwd()
print(f"The current working directory is: {current_directory}")

#env_system = os.environ
#print(env_system)

device is cpu
The current working directory is: c:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\Challenge-Sorbonne\modele_v13


In [7]:
# for Google Colab only
if 'COLAB_RELEASE_TAG' in os.environ:
    from google.colab import drive
    drive.mount('/content/drive')

In [19]:
current_directory = os.getcwd()
print(f"The current working directory is: {current_directory}")

# init du repertoire des données
if current_directory == '/content': # Google Colab
    #training_path_dir =   '/content/drive/MyDrive/Sorbonne_Data_challenge/folder_training_set'
    test_path_dir =       '/content/drive/MyDrive/Sorbonne_Data_challenge/folder_test_set'
    train_meta_data =     '/content/drive/MyDrive/Sorbonne_Data_challenge/training_set_metadata.csv'
    #file_split_training = '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge/split_training-colab.csv'

    model_save_file =     '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge-v12/model_sorbonne_weights.pth'
    filename_trained =    '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge_v12/list_hash_trained.csv'

    split_char = '/'
    test_init_file =        '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge-v12/test_set_to_predict.csv'
    filename_test_results = '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge-v12/test_prediction.csv'
    filename_test_res_partiels = '/content/drive/MyDrive/Colab Notebooks/Sorbonne-Challenge-v12/test_prediction_part.csv'

elif current_directory == r"c:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\Challenge-Sorbonne\modele_v13": #PC1
    split_char = '\\'
    #training_path_dir =   r"D:\ChallengeDST\folder_training_set"
    #training_path_dir = r'G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\Sorbonne_Data_challenge\folder_training_set'
    filename_trained = r"..\Pred_test_v13\list_hash_trained.csv"
    file_split_training = r'..\Pred_test_v13\split_training-pc-v12.csv'

    model_save_file =  r'..\Pred_test_v13\model_sorbonne_weights.pth'
    train_meta_data =  r'..\training_set_metadata.csv'

    #test_path_dir = r'G:\.shortcut-targets-by-id\19Ls71oZUG1aa1uTMu1laH6Nt3qK5gSZK\Sorbonne_Data_challenge\folder_test_set'
    test_path_dir = r"D:\ChallengeDST\folder_test_set\folder_test_set"
    #test_path_dir = r"..\..\Data\ChallengeSorbonne\training"
    test_init_file = r"..\test_set_to_predict.csv"
    filename_test_results = r"..\Pred_test_v13\test_prediction.csv"
    filename_test_res_partiels = r'..\Pred_test_v13\test_prediction_part.csv'

elif current_directory == r"c:\Users\jch_m\Documents Perso\DevPython\Challenge-sorbonne\modele_v13": #PC2
    split_char = '\\'

    training_path_dir =   r"D:\ChallengeDST\folder_training_set"
    #training_path_dir =   r"..\..\Data\ChallengeSorbonne\training"
    #training_path_dir =   r"..\..\Data\ChallengeSorbonne\training_s"
    filename_trained = r"..\modele_v11\list_hash_trained.csv"
    file_split_training = r'..\split_training-pc.csv'

    model_save_file =  r'..\Pred_test_v11_23K_2603\model_sorbonne_weights.pth'
    train_meta_data =  r'..\training_set_metadata.csv'

    test_path_dir =   r"D:\ChallengeDST\folder_training_set\folder_training_set"
    #test_path_dir = r"..\..\Data\ChallengeSorbonne\training"
    #test_path_dir = r"G:\Mon Drive\Colab Notebooks\Sorbonne-Challenge"
    test_init_file = r"..\test_set_to_predict.csv"
    filename_test_results = r"..\Pred_test_v11_23K_2603\test_prediction.csv"
    filename_test_res_partiels = r'..\Pred_test_v11_23K_2603\test_prediction_part.csv'
    filename_test_res_partiels = r'C:\Users\jch_m\OneDrive\test_prediction_part.csv'

else:
    print("**** ERROR: init files names and directories not done - root directory and environment not identified")

random_seed = 42

The current working directory is: c:\Users\jch_m\DocumentsPerso\Centrale-IAC\Cours\Python-ML\ProjectMaster\Challenge-Sorbonne\modele_v13


In [20]:

if os.path.exists(filename_test_res_partiels):
    df_pred_partiel = pd.read_csv(filename_test_res_partiels, sep=",")
    print("taille :",len(df_pred_partiel),filename_test_res_partiels)
    if len(df_pred_partiel)>0 :
      nan_count = df_pred_partiel['64bits'].isna().sum()
      print(f"Nombre de graphs predits: {len(df_pred_partiel) - nan_count}")
      print(f"Nombre de graphs A PREDIRE : {nan_count}")
      df_pred_partiel.tail()
    else:
      nan_count = df_pred_partiel['64bits'].isna().sum()
      print(f"Nombre de graphs predits: 0")
      print(f"Nombre de graphs A PREDIRE : tous")
else:
    print("pas predictions partielles trouvées, pas de fichier", filename_test_res_partiels)

taille : 220 ..\Pred_test_v13\test_prediction_part.csv
Nombre de graphs predits: 0
Nombre de graphs A PREDIRE : 220


# Init MODEL

In [26]:
# function to extract and tokenize the graph features
class AssemblyFeatureExtractor:
    def __init__(self):
        self.node_type_vocab = {}
        self.operation_vocab = {}
        self.register_vocab = {}
        self.memory_pattern_vocab = {}
        self.immediate_vocab = {}
        self.UNK_TOKEN = "<UNK>"
        self.max_registers_to_embed = 3  # Embed up to 3 registers

        # Regex patterns for parsing
        self.patterns = {
            'node_type': re.compile(r'([A-Z]+)\s*:'),
            'operation': re.compile(r':\s*([a-z]+)'),
            'registers': re.compile(r'(?:^|\s+)([a-z]{2,3}x|[a-z]{2,3}i|[a-z]{1,3}h|[a-z]{1,3}l)(?:$|\s+|,|\])'),
            'memory_ref': re.compile(r'\[(.*?)\]'),
            'immediate': re.compile(r'0x[0-9a-fA-F]+|\d+')
        }

        # Set to store all encountered registers for vocabulary building
        self.all_registers = set()

    def tokenize_line(self, line):
        tokens = {}
        if match := self.patterns['node_type'].search(line):
            tokens['node_type'] = match.group(1)
        if match := self.patterns['operation'].search(line):
            tokens['operation'] = match.group(1)
        if matches := self.patterns['registers'].findall(line):
            tokens['registers'] = [reg.lower() for reg in matches]
            self.all_registers.update(tokens['registers'])
        if match := self.patterns['memory_ref'].search(line):
            tokens['memory_ref'] = match.group(1)
        if match := self.patterns['immediate'].search(line):
            tokens['immediate'] = match.group(0)
        return tokens

    def extract_features(self, digraph):
        all_instruction_features = {}
        for node_id, node_data in digraph.nodes(data=True):
            label = node_data.get('label', '')
            instruction_features_list = []
            instructions = label.split('\n')
            for i, line in enumerate(instructions):
                tokens = self.tokenize_line(line)
                instruction_features = {}

                if 'operation' in tokens:
                    instruction_features['operation'] = tokens['operation']
                if 'node_type' in tokens:
                    instruction_features['node_type'] = tokens['node_type']
                if 'memory_ref' in tokens:
                    instruction_features['memory_ref'] = tokens['memory_ref']
                if 'immediate' in tokens:
                    instruction_features['immediate'] = tokens['immediate']

                # Binary features for register presence (before vocabulary is built)
                for reg in self.all_registers:
                    instruction_features[f'has_register_{reg}'] = 1 if 'registers' in tokens and reg in tokens['registers'] else 0

                # Position in the sequence
                instruction_features['position'] = i
                instruction_features_list.append(instruction_features)

            if len(instruction_features_list) == 1:
                all_instruction_features[node_id] = instruction_features_list[0]
            else:
                all_instruction_features[node_id] = instruction_features_list
        return all_instruction_features

    def build_vocabularies(self, input_data_list):
        node_types = set()
        operations = set()
        registers = set()
        memory_patterns = set()
        immediates = set()

        for item in input_data_list:
            if isinstance(item, nx.DiGraph):
                graph = item
                for _, node_data in graph.nodes(data=True):
                    label = node_data.get('label', '')
                    for line in label.split('\n'):
                        tokens = self.tokenize_line(line)
                        if 'node_type' in tokens:
                            node_types.add(tokens['node_type'])
                        if 'operation' in tokens:
                            operations.add(tokens['operation'])
                        if 'registers' in tokens:
                            registers.update(tokens['registers'])
                        if 'memory_ref' in tokens:
                            memory_patterns.add(tokens['memory_ref'])
                        if 'immediate' in tokens:
                            immediates.add(tokens['immediate'])
            elif isinstance(item, str):
                assembly_code = item
                for line in assembly_code.strip().split('\n'):
                    tokens = self.tokenize_line(line)
                    if 'node_type' in tokens:
                        node_types.add(tokens['node_type'])
                    if 'operation' in tokens:
                        operations.add(tokens['operation'])
                    if 'registers' in tokens:
                        registers.update(tokens['registers'])
                    if 'memory_ref' in tokens:
                        memory_patterns.add(tokens['memory_ref'])
                    if 'immediate' in tokens:
                        immediates.add(tokens['immediate'])

        self.node_type_vocab = {token: i + 1 for i, token in enumerate(sorted(list(node_types)))}
        self.node_type_vocab[self.UNK_TOKEN] = 0
        self.operation_vocab = {token: i + 1 for i, token in enumerate(sorted(list(operations)))}
        self.operation_vocab[self.UNK_TOKEN] = 0
        self.register_vocab = {token: i + 1 for i, token in enumerate(sorted(list(registers)))}
        self.register_vocab[self.UNK_TOKEN] = 0
        self.memory_pattern_vocab = {token: i + 1 for i, token in enumerate(sorted(list(memory_patterns)))}
        self.memory_pattern_vocab[self.UNK_TOKEN] = 0
        self.immediate_vocab = {token: i + 1 for i, token in enumerate(sorted(list(immediates)))}
        self.immediate_vocab[self.UNK_TOKEN] = 0

    def embed_instruction(self, instruction_features, embedding_dim=60):
        """Embeds a single assembly instruction feature dictionary."""
        embedding = []

        # Embed operation (if present)
        op_embedding = np.zeros(embedding_dim)
        if 'operation' in instruction_features and instruction_features['operation'] in self.operation_vocab:
            op_index = self.operation_vocab[instruction_features['operation']]
            op_embedding[op_index % embedding_dim] = 1.0  # Simple one-hot-like hashing
        embedding.extend(op_embedding)

        # Embed node type (if present)
        node_type_embedding = np.zeros(embedding_dim // 2)
        if 'node_type' in instruction_features and instruction_features['node_type'] in self.node_type_vocab:
            node_type_index = self.node_type_vocab[instruction_features['node_type']]
            node_type_embedding[node_type_index % (embedding_dim // 2)] = 1.0
        embedding.extend(node_type_embedding)

        # Embed registers (up to max_registers_to_embed)
        registers_present = [reg.split('_')[-1] for reg in instruction_features if reg.startswith('has_register_') and instruction_features[reg] == 1]
        register_embedding = np.zeros(self.max_registers_to_embed * (embedding_dim // 3)) # Allocate space
        for i, reg in enumerate(registers_present[:self.max_registers_to_embed]):
            if reg in self.register_vocab:
                reg_index = self.register_vocab[reg]
                register_embedding[i * (embedding_dim // 3) + (reg_index % (embedding_dim // 3))] = 1.0
        embedding.extend(register_embedding)

        # Embed memory pattern (if present)
        mem_embedding = np.zeros(embedding_dim // 2)
        if 'memory_ref' in instruction_features and instruction_features['memory_ref'] in self.memory_pattern_vocab:
            mem_index = self.memory_pattern_vocab[instruction_features['memory_ref']]
            mem_embedding[mem_index % (embedding_dim // 2)] = 1.0
        embedding.extend(mem_embedding)

        # Embed immediate (just presence for now)
        has_immediate = 1 if 'immediate' in instruction_features else 0
        embedding.append(has_immediate)

        # Position
        position = instruction_features.get('position', 0)
        embedding.append(position / 10.0) # Normalize position

        return np.array(embedding[:60]) # Truncate to max 60 dimensions

    def embed_graph(self, digraph):
        """Embeds all instructions within the nodes of a digraph."""
        embedded_nodes = {}
        raw_features = self.extract_features(digraph)
        for node_id, features in raw_features.items():
            if isinstance(features, list):  # Multiple instructions in a node
                embedded_nodes[node_id] = [self.embed_instruction(f) for f in features]
            else:  # Single instruction in a node
                embedded_nodes[node_id] = self.embed_instruction(features)
        return embedded_nodes

# Function to create dataloader
#
class AssemblyGraphDataset(Dataset):
    def __init__(self, graph_list, feature_extractor, hash_dim=512):
        self.graph_list = graph_list
        self.feature_extractor = feature_extractor
        self.hash_dim = hash_dim
        self.processed_data = self._process_all_graphs()

    def _process_all_graphs(self):
        processed_data = []
        for graph_item in self.graph_list: # Iterate through the list of graph items
            graph = graph_item # Access the NetworkX graph from the dictionary
            embedded_nodes = self.feature_extractor.embed_graph(graph)

            # Create PyTorch Geometric Data object
            node_features = []
            node_order = sorted(graph.nodes()) # Ensure consistent node order
            for node_id in node_order:
                embedding = embedded_nodes.get(node_id)
                if isinstance(embedding, list):
                    # For simplicity, if a node has multiple instructions, we take the first embedding
                    # You might want a more sophisticated way to handle this (e.g., averaging, using RNN)
                    node_features.append(embedding[0])
                elif embedding is not None:
                    node_features.append(embedding)
                else:
                    node_features.append(np.zeros(60)) # Handle missing embeddings

            # Get edges as a list of tuples
            edges = list(graph.edges())
            if not edges:
                edge_index = torch.empty((2, 0), dtype=torch.long)
            else:
                # Convert node labels to indices if they are not already numeric
                node_to_index = {node: i for i, node in enumerate(graph.nodes())}
                indexed_edges = [(node_to_index[u], node_to_index[v]) for u, v in edges]
                edge_index = torch.tensor(indexed_edges, dtype=torch.long).t().contiguous()

            # Dummy hash encoding (replace with your actual hash encoding)
            hash_encoded = torch.randn(len(node_features), self.hash_dim)

            # Convert lists of numpy arrays to single numpy arrays
            if isinstance(node_features, list):
                node_features_np = np.array(node_features)
            else:
                node_features_np = node_features  # Already a numpy array

            # Convert the numpy arrays to torch tensors
            node_features_tensor = torch.tensor(node_features_np, dtype=torch.float)

            data = Data(x=node_features_tensor,
                        edge_index=edge_index,
                        hash_encoded=hash_encoded)
            processed_data.append(data)

        return processed_data

    def __len__(self):
        return len(self.processed_data)

    def __getitem__(self, idx):
        return self.processed_data[idx]
def collate_fn(batch):
    """Collates a list of PyTorch Geometric Data objects into a Batch object.
    Args:
        batch (list): A list of `torch_geometric.data.Data` objects.
    Returns:
        torch_geometric.data.Batch: A `Batch` object that combines the data
            objects in the `batch`.
    """
    return Batch.from_data_list(batch)

# Define the AssemblyGAT model
class AssemblyGAT(nn.Module):
    def __init__(self, node_feature_dim, hidden_dim=64, output_dim=453, heads=8, dropout=0.6, hash_dim=512):
        super(AssemblyGAT, self).__init__()
        self.conv1 = GATConv(node_feature_dim, hidden_dim, heads=heads, dropout=dropout)
        self.conv2 = GATConv(hidden_dim * heads, output_dim, heads=1, concat=False, dropout=dropout)
        self.dropout = dropout
        self.hash_dim = hash_dim

    def forward(self, data):
        """Forward pass through the GAT model"""
        x, edge_index, hash_encoded, batch = data.x, data.edge_index, data.hash_encoded, data.batch

        # First GAT layer with activation and dropout
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=self.dropout, training=self.training)

        # Second GAT layer
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)
        return x

#######################################
# Function to parse digraph string (as in previous example)
def parse_digraph_string(digraph_str):
    """Parse the digraph string to create a NetworkX DiGraph"""
    # Simple parser for the provided format
    G = nx.DiGraph()

    # Regular expressions for node and edge definitions
    node_pattern = re.compile(r'"([^"]+)"\s*\[label\s*=\s*"([^"]+)"\]')
    edge_pattern = re.compile(r'"([^"]+)"\s*->\s*"([^"]+)"')

    # Extract nodes and edges
    for line in digraph_str.strip().split('\n'):
        line = line.strip()

        # Skip the Digraph G { and } lines
        if line == 'Digraph G {' or line == '}':
            continue

        # Parse node definitions
        node_match = node_pattern.search(line)
        if node_match:
            node_id = node_match.group(1)
            label = node_match.group(2)
            G.add_node(node_id, label=label)
            continue

        # Parse edge definitions
        edge_match = edge_pattern.search(line)
        if edge_match:
            source = edge_match.group(1)
            target = edge_match.group(2)
            G.add_edge(source, target)
    return G

def process_batch(file_batch):
    """
    Processes a batch of files.
    Args:
        file_batch (list): A list of file paths.
    """
    graph_list_curr = []
    list_hash = []
    file_with_err = 0

    for full_path_file in file_batch:
        #print(full_path_file)
        try:
            if os.path.exists(full_path_file):
                file_size = os.path.getsize(full_path_file)
                if file_size/1000 > 100_000: #file too big
                    print("size", file_size/1000, full_path_file)
                    continue
            
            with open(full_path_file, 'r') as f:
                hash_file = full_path_file.split('.jso')[0]
                hash_file = hash_file.split(split_char)[-1]

                # test if file content has no error tag
                digraph_str = f.read()
                if 'ERROR' in digraph_str:
                    file_with_err += 1
                    continue
                f.close()
                G = parse_digraph_string(digraph_str)
                graph_list_curr.append(G)
                list_hash.append(hash_file)

        except FileNotFoundError:
            print(f"Error: File not found: {full_path_file}")
        except Exception as e:
            print(f"Error processing {full_path_file}: {e}")

    return graph_list_curr, list_hash

def get_metadata_from_hash(hash_list):
    hash_meta_values = []
    for hash_name in hash_list:
        df_y_values = df_meta_train[df_meta_train['name'] == hash_name].copy()
        df_y_values.drop(columns=['name'], inplace=True)
        if len(df_y_values) > 0 :
             hash_meta_values.append(df_y_values.values[0]) # Get the first row's values
        else:
            print(f"{hash_name} not found in metadata")
    return hash_meta_values



# MODE TEST ONLY

Generate prediction based on the requested list of hash (no true labels available)


In [27]:
df_test_init = pd.read_csv(test_init_file, sep=",")
col_to_predict = df_test_init.columns
df_test_init.head()
print(f"Test init: {df_test_init.shape[1]-1} labels to predict for {df_test_init.shape[0]} graphs \nfrom files located: {test_path_dir}\n")

Test init: 453 labels to predict for 3000 graphs 
from files located: D:\ChallengeDST\folder_test_set\folder_test_set



In [ ]:
#
#  Execution de la boucle de predictions
#
# init test
df_all_labels = df_test_init
list_test_hash = []

# Si fichier de predictions partielles disponible
if os.path.exists(filename_test_res_partiels):
    print("Predictions partielles disponibles")
    df_pred_partiel = pd.read_csv(filename_test_res_partiels, sep=",")
    if len(df_pred_partiel)>0 :
      nan_count = df_pred_partiel['64bits'].isna().sum()
      init_idx_pred = df_test_init.shape[0] - nan_count # initialise la boucle du 1er graph non predit
      indices_a_mettre_a_jour = df_all_labels.index[:init_idx_pred]
      print(f"Nombre de graphs predits: {init_idx_pred}")
      print(f"Nombre de graphs A PREDIRE : {nan_count}")

      # Mettre à jour les valeurs des 25 premières lignes de df_pred avec celles de df_pred_partiel
      df_pred_partiel_head = df_pred_partiel.head(init_idx_pred)
      if len(df_pred_partiel_head) == len(indices_a_mettre_a_jour):
        df_all_labels.loc[indices_a_mettre_a_jour] = df_pred_partiel_head.values

    else:
      nan_count = df_test_init.shape[0]
      init_idx_pred = 0
      print(f"Fichier dispo, mais Nombre de graphs predits: 0")
      print(f"Nombre de graphs A PREDIRE : {df_test_init.shape[0]}")

# Si pas de fichier de predictions partielles disponible
else:
    print("Pas de fichier de prediction partielle disponible")
    print(f"Nombre de graphs A PREDIRE : {df_test_init.shape[0]}")
    nan_count = df_test_init.shape[0]
    init_idx_pred = 0  # initialise la boucle du 1er graph non predit

batch_data_size = 1
batch_files_size = 1
node_feature_dim = 60  #dim=20 pour V12, dim=15 pour V11

num_files = len(df_all_labels)
nb_batch = (num_files // batch_files_size) + 1

extractor = AssemblyFeatureExtractor()
model = AssemblyGAT(node_feature_dim=node_feature_dim, hidden_dim=64, output_dim=453)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

model.load_state_dict(torch.load(model_save_file, weights_only=True))
model.eval()
files_test_list = [os.path.join(test_path_dir,f) for f in os.listdir(test_path_dir) if f.endswith(".json")]
with torch.no_grad():
    # pour tous les hash du fichiers test
    print(f"**** Dataset Processing for Test")
    count_batch = 0
    total_batch = round((len(df_test_init)-init_idx_pred)/batch_files_size , 0)

    # on commence la boucle à l'index definit au dessus
    for i in range(init_idx_pred, num_files, batch_files_size):
        #batch_files = df_all_labels[i:i + batch_files_size]
        batch_files = files_test_list[i:i + batch_files_size]
        batch_num = i // batch_files_size + 1
        print(f"Processing batch {batch_num}/{nb_batch}: {len(batch_files)} files")

        batch_graph_list, hash_t = process_batch(batch_files)
        if not batch_graph_list:
            print("graph list is empty for this batch")
            continue
        
        #print(hash_t)
        hash_y_list = df_all_labels[df_all_labels['name'].isin(hash_t)]
        #print(hash_y_list)
        
        # Create the AssemblyGraphDataset # Create the DataLoader
        extractor.build_vocabularies(batch_graph_list)
        assembly_dataset = AssemblyGraphDataset(batch_graph_list, extractor)
        dataloader = DataLoader(assembly_dataset, batch_size=batch_data_size, shuffle=False, collate_fn=collate_fn)

        for batch in dataloader:
            batch = batch.to(device)

            # Forward pass
            output = model(batch)

            predicted_labels_prob = torch.sigmoid(output)
            prediction_labels = (predicted_labels_prob > 0.5).int()
        
            if len(prediction_labels) != len(hash_y_list):
                print(f"*** error: len(prediction_labels)={len(prediction_labels)}, len(hash_names)={len(hash_y_list)}")

            for i, hash_val in enumerate(hash_y_list['name']):
                row_index = df_all_labels[df_all_labels['name'] == hash_val].index
                if (not row_index.empty) and (i < len(prediction_labels)):

                    label_list = prediction_labels[i].tolist()  # Convert tensor row to list
                    int_label_list = [int(val) for val in label_list]  # Convert to int
                    # Update the DataFrame
                    for j in range(len(int_label_list)):
                        df_all_labels.iloc[row_index, j+1] = int_label_list[j]
                else:
                    print(f"Warning: hash_val '{hash_val}' not found in df_all_labels.")

        count_batch += 1
        # on sauve les results partiels
        df_all_labels.to_csv(filename_test_res_partiels, header=True ,index=False )

    print(f"Test end: nb pred labels: {len(df_all_labels)} ")
    df_all_labels.to_csv(filename_test_results, header=True ,index=False )

Pas de fichier de prediction partielle disponible
Nombre de graphs A PREDIRE : 3000
**** Dataset Processing for Test
Processing batch 1/3001: 1 files
Processing batch 2/3001: 1 files
Processing batch 3/3001: 1 files
Processing batch 4/3001: 1 files
Processing batch 5/3001: 1 files
Processing batch 6/3001: 1 files
Processing batch 7/3001: 1 files
Processing batch 8/3001: 1 files
Processing batch 9/3001: 1 files
Processing batch 10/3001: 1 files
Processing batch 11/3001: 1 files
Processing batch 12/3001: 1 files
Processing batch 13/3001: 1 files
Processing batch 14/3001: 1 files
Processing batch 15/3001: 1 files
Processing batch 16/3001: 1 files
Processing batch 17/3001: 1 files
Processing batch 18/3001: 1 files
Processing batch 19/3001: 1 files
Processing batch 20/3001: 1 files
Processing batch 21/3001: 1 files
Processing batch 22/3001: 1 files
Processing batch 23/3001: 1 files
Processing batch 24/3001: 1 files
Processing batch 25/3001: 1 files
Processing batch 26/3001: 1 files
Processi

In [1]:
print(df_all_labels.shape)
df_all_labels.head(10)

NameError: name 'df_all_labels' is not defined